<a href="https://colab.research.google.com/github/God-Orcale/AI_Test/blob/main/Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
batch_size = 64
learning_rate = 0.001
num_epochs = 10

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
train_dataset = torchvision.datasets.MNIST(root='./data',train=True,transform=transform,download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',train=False,transform=transform,download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.60MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.16MB/s]


In [4]:
class SimpleNN(nn.Module):
  def __init__(self):
    super(SimpleNN,self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(28*28,128)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(128,64)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(64,10)
  def forward(self,x):
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    x = self.relu2(x)
    x = self.fc3(x)
    return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [6]:
def train_one_epoch(model,dataloader,criterion,optimizer,device):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0
  for images,labels in dataloader:
    images,labels = images.to(device),labels.to(device)
    outputs = model(images)
    loss = criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()*images.size(0)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  epoch_loss = running_loss/len(dataloader.dataset)
  epoch_acc = correct/total
  return epoch_loss,epoch_acc


In [7]:
def evaluate(model,dataloader,criterion,device):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0
  with torch.no_grad():
    for images,labels in dataloader:
      images,labels = images.to(device),labels.to(device)
      outputs = model(images)
      loss = criterion(outputs,labels)
      running_loss += loss.item()*images.size(0)
      _,predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  epoch_loss = running_loss/len(dataloader.dataset)
  epoch_acc = correct/total
  return epoch_loss,epoch_acc

In [8]:
for epoch in range(num_epochs):
  train_loss,train_acc = train_one_epoch(model,train_loader,criterion,optimizer,device)
  test_loss,test_acc = evaluate(model,test_loader,criterion,device)
  print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

Epoch 1/10, Train Loss: 0.2726, Train Acc: 0.9186, Test Loss: 0.1380, Test Acc: 0.9570
Epoch 2/10, Train Loss: 0.1116, Train Acc: 0.9665, Test Loss: 0.1041, Test Acc: 0.9679
Epoch 3/10, Train Loss: 0.0801, Train Acc: 0.9753, Test Loss: 0.1031, Test Acc: 0.9664
Epoch 4/10, Train Loss: 0.0612, Train Acc: 0.9810, Test Loss: 0.0883, Test Acc: 0.9718
Epoch 5/10, Train Loss: 0.0488, Train Acc: 0.9840, Test Loss: 0.0936, Test Acc: 0.9709
Epoch 6/10, Train Loss: 0.0394, Train Acc: 0.9870, Test Loss: 0.0954, Test Acc: 0.9739
Epoch 7/10, Train Loss: 0.0360, Train Acc: 0.9874, Test Loss: 0.0789, Test Acc: 0.9781
Epoch 8/10, Train Loss: 0.0272, Train Acc: 0.9910, Test Loss: 0.0948, Test Acc: 0.9734
Epoch 9/10, Train Loss: 0.0259, Train Acc: 0.9913, Test Loss: 0.1042, Test Acc: 0.9743
Epoch 10/10, Train Loss: 0.0229, Train Acc: 0.9920, Test Loss: 0.1131, Test Acc: 0.9729
